1, click run all to run this code

2, it take 1~2 hour to finish 

In [1]:
# used to find spark path, please change path to your spark install path
try:
    import findspark
    findspark.init('/home/mark/spark-2.2.1-bin-hadoop2.7/')
except:
    pass 

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

from pyspark.sql import Row
from pyspark.sql.functions import length,rand
import time

In [3]:
start=time.time()

## Question 1 : find dog/cat owner

' The criterion of filtering dog or cat owners is that their comments containing word cat or dog. This assumption is not 
rigorious since the user may just metioned cat or dog by chance. There are two reason i choose this criterion: 
1, It give fairly amount of cat/dog owner, about 1/11 total of data. 
2, For business intuition, there is a high probability that majority of these users are interested on topic related with cat or dog. 

   A more reasonable approach should using a iterative approach. For each interation, using set of key word from classifier of dog/cat owner as criterion to pick up training data labled as cat/dog until the set of key words became stable. ' 

In [4]:
spark=SparkSession.builder.appName('animal').config('spark.master','local').getOrCreate()

In [5]:
data=spark.read.csv('animals_comments.csv',inferSchema=True,header=True)

In [6]:
print('There are total {} comments.'.format(data.count()))

There are total 5820035 comments.


In [7]:
data.show()

+--------------------+------+--------------------+
|        creator_name|userid|             comment|
+--------------------+------+--------------------+
|        Doug The Pug|  87.0|I shared this to ...|
|        Doug The Pug|  87.0|  Super cute  😀🐕🐶|
|         bulletproof| 530.0|stop saying get e...|
|       Meu Zoológico| 670.0|Tenho uma jiboia ...|
|              ojatro|1031.0|I wanna see what ...|
|     Tingle Triggers|1212.0|Well shit now Im ...|
|Hope For Paws - O...|1806.0|when I saw the en...|
|Hope For Paws - O...|2036.0|Holy crap. That i...|
|          Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
|       Brian Barczyk|2698.0|Call the teddy Larry|
|            The Dodo|2702.0|  😐🤔😓😢😭😭😭😭😟|
|Hope For Paws - O...|2911.0|That mother cat l...|
|Hope For Paws - O...|2911.0|Its people like H...|
|   Talking Kitty Cat|2911.0|steve: No wet foo...|
|    Brave Wilderness|3224.0|Dont call this a ...|
|          MaxluvsMya|3267.0|why are you alway...|
|Rise Up Society F...|3372.0|          Deb 

In [8]:
# convert dataframe to rdd 
datardd=data.rdd


In [9]:
# filter out data without creator_name or userid or comment

# data without creator name or userid or comment
df_anomal=datardd.filter(lambda x: (x.creator_name==None)or (x.userid==None)or (x.comment==None))

# normal data with creator_name and userid and comment 
df_normal=datardd.filter(lambda x:  (x.creator_name!=None) and (x.userid!=None) and (x.comment!=None))

In [10]:
# intution: sometimes, one used may be post several comments under same creator_name channel. 
# so i conbined the comment of same user under same channel with reduce key function, key-pair is ( userid, comments)
# x[1] is userid   x[2] is comment

df_combine=df_normal.map(lambda x: ((x[1]),x[2])).   \
                     reduceByKey(lambda x,y: x+y)

In [11]:
df_combine.count()

2524849

In [12]:
#df_combine.take(5)

In [13]:
def add_cate(arr):
    ''' use key word select cat/dog owner
    assumption: The cat/dog owner has keyword [cat, dog] on their comment
    '''
    
    tem={}
#    tem['creator_name']=arr[0][0]
#    tem['userid']=arr[0][1]
    tem['userid']=arr[0]
    
    comment=arr[1].strip().lower().split()     # cleaning comment data 
    clean_comment=[ x for x in comment if x.isalpha()]  # remove digit, puncuation 
    
    tem['combined_comment']=(' ').join(clean_comment)
    if 'dog' in (tem['combined_comment']):
        tem['category']='cat_dog'
        return tem
    elif 'cat' in (tem['combined_comment']):
        tem['category']='cat_dog'
        return tem
    else:
        tem['category']='other'
        return tem

In [14]:
# use add_cate function to add label [cat_dog, other] on data and clean_comment 

df_cate=df_combine.map(lambda x:Row(**add_cate(x)))

In [15]:
#df_tem=df_combine.map(lambda x:Row(**add_cate(x)))

In [16]:
print('There are total {} userid now.'.format(df_cate.count()))


There are total 2524849 userid now.


In [17]:
# convert rdd to dataframe 
new_frame=df_cate.toDF()

In [18]:
# each userid only have one combine_comment 
new_frame.show()

+--------+--------------------+---------+
|category|    combined_comment|   userid|
+--------+--------------------+---------+
|   other|subscribed he is ...|2097155.0|
|   other|noooooooooooooooo...|1048580.0|
|   other|the christmas mus...|      5.0|
| cat_dog|what do you do wi...|2097160.0|
|   other|      вы лучьше всех|1048585.0|
|   other|       i hope hes ok|     10.0|
|   other|made me go rewatc...|2097165.0|
|   other|how many times di...|1048590.0|
|   other|i guess im a hedg...|     15.0|
|   other|this channel is s...|2097170.0|
|   other|people are heartless|1048595.0|
|   other|explains my depre...|     20.0|
|   other|at you can see a ...|2097175.0|
| cat_dog|he speaks german ...|1048600.0|
|   other|                    |     25.0|
|   other|its never gonna h...|2097180.0|
|   other| use your name brian|1048605.0|
|   other|                    |     30.0|
|   other|    what gun are you|2097185.0|
|   other|my dick hurts whi...|1048610.0|
+--------+--------------------+---

# Comment:
 Now, each userid only have feature as combined_comments,

## Question 2: build classifier

' Build classifier: 
  data_cleaning_process (add_cate function): strip blank space, convert all words into lower, remove all none_letter,
  add new feature: length of comments. The intution behind this is that ususally the comments of the dog/cat owner have more word on comment.
  using Logistic classifier. 

In [19]:
# intuition: ususally the comments of the dog/cat owner have more word on comment.
# so create a new feather as length of comment

new_frame=new_frame.withColumn('length',length(new_frame['combined_comment']))
new_frame.groupby('category').mean().show()

+--------+-----------------+------------------+
|category|      avg(userid)|       avg(length)|
+--------+-----------------+------------------+
|   other|1271203.735342072|101.08419325590432|
| cat_dog|1269302.992542195| 522.0088307599414|
+--------+-----------------+------------------+



In [20]:
# data frame with label cat_dog
df_dog_cat=new_frame.filter((new_frame['category']=='cat_dog') )
# data_frame with label other
other=new_frame.filter((new_frame['category']=='other') )

In [21]:
print('There are {} userid could be dog/cat owner.'.format(df_dog_cat.count()))

There are 278232 userid could be dog/cat owner.


In [77]:
# since the data is extremly unbalanced, so select 1:1 ratio of labeled dog_cat and other data
# because there are total 3176574 comments, and 
# about 319996 related with cat/dog. The ratio is roughly 10:1
# so random split rest (data not labeled cat_dog) into 9:1 ratio, 
# and combine those 10% with data labeled as cat_dog  to build training data 

unlabeled9,unlabeled1=other.randomSplit([0.7,0.3])

data=df_dog_cat.unionAll(unlabeled1).orderBy(rand())

In [78]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF,Tokenizer,IDF,VectorAssembler
from pyspark.ml.feature import StringIndexer,StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.sql import Column
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [79]:
#data.registerTempTable('data')

In [80]:
# combine creator and comment to build a new text feature 

#new_dog_cat=spark.sql("select *, \
#                  CONCAT(creator_name,' ',comment) as cnc from data")
new_dog_cat=data

In [81]:
# encode the category
encode=StringIndexer(inputCol='category',outputCol='label')
# tokenization 
token_cnc=Tokenizer(inputCol='combined_comment',outputCol='cnc_token')
# remove stopwords
stopremover=StopWordsRemover(inputCol='cnc_token',outputCol='stop_remove')
# build term frenquence 
cnc_TF=HashingTF(inputCol='stop_remove',outputCol='cnc_tf')
# build term frenquence inverse document frequence
idf=IDF(inputCol='cnc_tf',outputCol='cnc_tfidf')

vectorass=VectorAssembler(inputCols=['cnc_tfidf','length'],outputCol='features')

# build pipeline for data processisng 
processing_pipeline=Pipeline(stages=[encode,token_cnc,stopremover,cnc_TF,idf,vectorass])

In [82]:
# used build pipeline processing data 
processor=processing_pipeline.fit(new_dog_cat)

processed_data=processor.transform(new_dog_cat)

In [83]:
clean_data=processed_data.select('label','features')

In [84]:
a=processed_data.limit(5)

In [85]:
a.cache()

DataFrame[category: string, combined_comment: string, userid: double, length: int, label: double, cnc_token: array<string>, stop_remove: array<string>, cnc_tf: vector, cnc_tfidf: vector, features: vector]

In [86]:
b=a.select('category','label').distinct().rdd.take(5)

In [87]:
dict_encode={}
for i in range(len(b)):
    dict_encode[b[i].category]=b[i].label


In [88]:
# split data into train and test part
train,test=clean_data.randomSplit([0.7,0.3])

# build classifier
#nb=LogisticRegression(featuresCol='features')
nb=NaiveBayes(featuresCol='features')


clf=nb.fit(train)

test_result=clf.transform(test)

In [89]:
test_result.cache()
#spark.catalog.clearCache()

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [90]:
# build evaluate metrics 
binary_eval=BinaryClassificationEvaluator()

acc=binary_eval.evaluate(test_result)

In [91]:
print('The area under of ROC curve is {0:.2f}.'.format(acc))

The area under of ROC curve is 0.30.


In [92]:
tp=test_result[(test_result['label']==dict_encode['cat_dog'])& (test_result['prediction']==dict_encode['cat_dog'])].count()
tn=test_result[(test_result['label']==dict_encode['other'])& (test_result['prediction']==dict_encode['other'])].count()
fp=test_result[(test_result['label']==dict_encode['other'])& (test_result['prediction']==dict_encode['cat_dog'])].count()
fn=test_result[(test_result['label']==dict_encode['cat_dog'])& (test_result['prediction']==dict_encode['other'])].count()

print('--------Confusion matrix---------')
print('The true positive is {}.'.format(tp))
print('The false positive is {}.'.format(fp))
print('The false negative is {}.'.format(fn))
print('The true negative is {}.'.format(tn))

--------Confusion matrix---------
The true positive is 80954.
The false positive is 93040.
The false negative is 2631.
The true negative is 109292.


In [93]:
print('---------Analysis----------')

print('The accuracy is {0:.2f}.'.format((tp+tn)/(tp+tn+fp+fn)))
print('The true positive rate is {0:.2f}'.format(tp/(tp+fn)))
print('The false positive rate is {0:.2f}.'.format(fp/(fp+tn)))
print('The false negative rate is {0:.2f}.'.format(fn/(fn+tp)))

---------Analysis----------
The accuracy is 0.67.
The true positive rate is 0.97
The false positive rate is 0.46.
The false negative rate is 0.03.


Comment: i have applied Logistic and NaiveBayes model, The logistic regression have give area under ROC curve 0.91, while NaiveBayes give area under ROC curve 0.72. since process of labeling data is not accuracy. I prefer NB over logistic. Because 0.91 accuracy 
    is not reasonable based on  my label processing. The furture should build a more sophiscate way to label the data.


## Quesntion 3

Appling classfier on all the data

In [94]:
#new_frame.show(5)

In [95]:
new_frame.registerTempTable('all_data')

In [96]:
#all_category=spark.sql("select *, \
#                  CONCAT(creator_name,' ',comment) as cnc from all_data")
all_category=new_frame

In [97]:
# using build pipeline to process all data 

processed_all_data=processor.transform(all_category)

In [98]:
# apply classfier to all the data 

test_result=clf.transform(processed_all_data)

In [99]:
#test_result.show()

In [100]:
#all_data=test_result.select('creator_name','userid','comment','prediction')
all_data=test_result.select('userid','combined_comment','prediction')

In [103]:
# filter out category with cat/dog
all_data_dog_cat=all_data.filter(all_data['prediction']==dict_encode['cat_dog'])
all_data_other=all_data.filter(all_data['prediction']==dict_encode['other'])

In [104]:
# while the true positive rate is 0.67. so the number of dog/cat owner is number of predicted dog/cat owner multiple 
# true positive rate 
dog_cat_count=all_data_dog_cat.count()
print('There are total {} predicted dog/cat owner.'.format(dog_cat_count))

There are total 1348834 predicted dog/cat owner.


There are total 2524849 userid

Under this situation, the predicted cat/dog owner is 1348834 for traing data with 3:1 ratio (3 other vs 1 cat/dog owner).
While for 1:1 ratio, the predicted cat/dog owner is about 1551838

The above analysis are based on pre-labeled training data, which is not rigorous. This process could be furture proved by 
repeatly analyze the words set for pre-labeling traing data. 

## Question4 : find topic of dog/cat owner
  Using LDA model for topic analysis:
  1, choose cat_dog owner 
  2, data preparing: strip black space, convert work to lower, remove none_letter word, remove word that we are not interested, like length of word smaller than 4 or 'that','this' et al.,

In [49]:
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer

In [50]:
# convert all dog/cat comment to rdd
comment=all_data_dog_cat.select('combined_comment').rdd

comment_other=all_data_other.select('combined_comment').rdd

In [51]:
# clean comment for topic analysis
# set of word to remove before LDA analysis, collected after each LDA analysis
stopword=['this','that','what','these','those','with','such','they','just',  \
         'would','them','your','from','have','thats','dont','thats','will',  \
         'really','there','about','please','more','much','still','being',   \
         'other','little','some','video','videos','could','wont','even','very',  \
         'back','even','always','like','love','good','nice','because','into',   \
         'were','their','is','are','great','then','when','been','know','want','think',  \
         'make','should']     

def convert_line_to_Row(line):
    tem={}
    tem['combined_comment']=line
    return tem
# comment of dog and cat owner
data0=comment.map(lambda line: line['combined_comment'].strip().lower()).  \
              map(lambda line: line.split()).                      \
              map(lambda line: [x for x in line if x.isalpha()]).  \
              map(lambda line: [x for x in line if len(x)>3]).     \
              map(lambda line: [x for x in line if x not in stopword]).    \
            filter(lambda line: line!=[]).map( lambda line: ' '.join(line)).  \
                map(lambda line : Row(**convert_line_to_Row(line)))

                # comment of other user
data0_other=comment_other.map(lambda line: line['combined_comment'].strip().lower()).  \
              map(lambda line: line.split()).                      \
              map(lambda line: [x for x in line if x.isalpha()]).  \
              map(lambda line: [x for x in line if len(x)>3]).     \
              map(lambda line: [x for x in line if x not in stopword]).    \
            filter(lambda line: line!=[]).map( lambda line: ' '.join(line)).  \
                map(lambda line : Row(**convert_line_to_Row(line)))

In [52]:
# convert processed comment into dataframe
data1=data0.toDF()

data1_other=data0_other.toDF()

In [53]:
# tokenize processed comment 
token_cnc=Tokenizer(inputCol='combined_comment',outputCol='comment_token')
data2=token_cnc.transform(data1)

data2_other=token_cnc.transform(data1_other)
# remove stop words
stopremove=StopWordsRemover(inputCol='comment_token',outputCol='stop_remover')
data3=stopremove.transform(data2)

data3_other=stopremove.transform(data2_other)

#convert them into term frquency 
countvect=CountVectorizer(inputCol='comment_token',outputCol='features')

countmodel=countvect.fit(data3)
data4=countmodel.transform(data3)

countmodel_other=countvect.fit(data3_other)
data4_other=countmodel_other.transform(data3_other)

In [54]:
# parameter of LDA model
number_topic=3
max_number_iteration=20

# Using lda model to train topic, assuming for 3 topic

def print_topic(a,number_topic,tf_model):
    '''
    input: topics is the output of describeTopics()
    '''

    b=a.head(number_topic)
    for i in range(number_topic):
        topics1=[]
        for j in b[i]['termIndices']:
            topics1.append(tf_model.vocabulary[j])
        print('The number {} topics: {}'.format(i+1,topics1))

def lda_model(dataframe,number_topic,max_number_iteration,tf_model):

    '''LDA model '''
    ldf=LDA(k=number_topic,featuresCol='features',maxIter=max_number_iteration,seed=1)
    ldf_model=ldf.fit(dataframe)

    topics=ldf_model.describeTopics()

    a=topics.select('termIndices')
    print_topic(a,number_topic,tf_model)

In [55]:
# for dog/cat user 
print('Topic for dog/cat owner')
lda_model(data4,number_topic,max_number_iteration,countmodel)

Topic for dog/cat owner
The number 1 topics: ['people', 'animals', 'keep', 'thank', 'cute', 'time', 'hope', 'going', 'also', 'cant']
The number 2 topics: ['fucking', 'fuck', 'jesus', 'cute', 'funny', 'shit', 'name', 'keep', 'call', 'ever']
The number 3 topics: ['gohan', 'dogs', 'poor', 'husky', 'birthday', 'happy', 'name', 'kitty', 'people', 'sylvester']


For dog/cat owner, the key words of topic 1 is about 'people', 'cute', it's more like praising. Topics 3 catched key words 'kitty' and 'husky'. I assume husky is a popular dog breed among dog users.  The key word 'birthday' was shown up with 'dogs'/'kitty'. The birthday Topic of these cat/dog owner indicated some presents, like toy, could be a good recommentation item each for creator of channel.    

In [56]:
print('Topic for other owner')
lda_model(data4_other,number_topic,max_number_iteration,countmodel_other)

Topic for other owner
The number 1 topics: ['coyote', 'name', 'cute', 'raptor', 'peterson', 'best', 'looks', 'stung', 'cool', 'channel']
The number 2 topics: ['mạnh', 'durian', 'clip', 'benim', 'gecko', 'chơi', 'muhabbet', 'fruit', 'mình', 'shout']
The number 3 topics: ['para', 'quiero', 'pero', 'como', 'esta', 'creo', 'tiene', 'funko', 'este', 'marvel']


The key word dog/cat do not show on topic of other user. The 'coyote','raptor' and 'stung' are definitely not words related with dog/cat topic. Topic 3 catched a quite few Spanish words. So number of Spanish user should be most beside English-Speaking user.  

## Question 5:

In [57]:
all_data.registerTempTable('all_data')

In [58]:
all_data.show()

+---------+--------------------+----------+
|   userid|    combined_comment|prediction|
+---------+--------------------+----------+
|2097155.0|subscribed he is ...|       0.0|
|1048580.0|noooooooooooooooo...|       0.0|
|      5.0|the christmas mus...|       0.0|
|2097160.0|what do you do wi...|       0.0|
|1048585.0|      вы лучьше всех|       1.0|
|     10.0|       i hope hes ok|       0.0|
|2097165.0|made me go rewatc...|       0.0|
|1048590.0|how many times di...|       0.0|
|     15.0|i guess im a hedg...|       0.0|
|2097170.0|this channel is s...|       0.0|
|1048595.0|people are heartless|       0.0|
|     20.0|explains my depre...|       0.0|
|2097175.0|at you can see a ...|       0.0|
|1048600.0|he speaks german ...|       0.0|
|     25.0|                    |       1.0|
|2097180.0|its never gonna h...|       0.0|
|1048605.0| use your name brian|       0.0|
|     30.0|                    |       1.0|
|2097185.0|    what gun are you|       0.0|
|1048610.0|my dick hurts whi...|

In [59]:
normal_data=df_normal.toDF()
normal_data.registerTempTable('normal_data')

In [60]:
# group data by creator_name, since cat_dog are labeled as 1.0, so sum(prediction) is number of total cat_dog owner for 
# each under each creator_name. and count(distinct_userid) are number of total userid. 

#ad=spark.sql('select creator_name, count(distinct userid) as total, sum(prediction) as cat_dog  \
#       from all_data group by creator_name')

#ad=spark.sql('select n.creator_name, n.userid, a.prediction from normal_data as n inner join  \
#             all_data as a  on n.userid==a.userid')
ad=spark.sql('select n.creator_name, n.userid, a.prediction from \
            (select creator_name, userid from normal_data group by creator_name, userid) as n inner join  \
            all_data as a  on n.userid==a.userid')

In [61]:
#ad.show()

In [62]:
ad.registerTempTable('ad')

In [63]:
ad1=spark.sql('select creator_name, count(distinct userid) as total, (count(distinct userid)-sum(prediction)) as cat_dog  \
       from ad group by creator_name')

In [64]:
# creator_name orderby  percentage of cat_dog owner
# i filterd creator_name with more than 100 userid
Percentage_of_user=ad1.withColumn('percentage',ad1['cat_dog']/ad1['total']).sort('percentage',ascending=False).  \
                                 filter(ad1['total']>50)

In [65]:
Percentage_of_user.cache()

DataFrame[creator_name: string, total: bigint, cat_dog: double, percentage: double]

In [66]:
# print top 10 creator_name with highest percentage of cat/dog owner

number_rank=50
print('Creator_name with number of userid more than 50')
print('The top {} creator_name are {}.'.format(number_rank,Percentage_of_user.limit(number_rank).show()))

Creator_name with number of userid more than 50
+--------------------+-----+-------+------------------+
|        creator_name|total|cat_dog|        percentage|
+--------------------+-----+-------+------------------+
|Dragon Tamer Rept...|   59|   59.0|               1.0|
|       Day5 Aquatics|   92|   91.0|0.9891304347826086|
|         Mike Hughes|   74|   73.0|0.9864864864864865|
|    coloradoreinsman|   60|   59.0|0.9833333333333333|
|       Brent Atwater|  263|  258.0|0.9809885931558935|
|    Dr. Karen Becker|  132|  129.0|0.9772727272727273|
|             Meldium|   84|   82.0|0.9761904761904762|
|         Joey Ferris|  199|  194.0|0.9748743718592965|
|    Maggie Krukowski|   68|   66.0|0.9705882352941176|
|   Calm My Dog House|   99|   96.0|0.9696969696969697|
|      Life With Dogs|   65|   63.0|0.9692307692307692|
|Daves Pacific Nor...|  162|  157.0|0.9691358024691358|
|           RatGirl44|  284|  275.0|0.9683098591549296|
|Sanders Slitherin...|   63|   61.0|0.9682539682539683|


## Comment:
    For creator_name have more than 50 userid. The creator_nane like 'ServiceDog Vlog','Calm My Dog House','Life With Dogs' et al are related with dog/cat literaly. There are a few creator_name with key word of 'fish' and 'aquatics' also were classified as dog/cat owner. I assume the comment of 'fish' and 'dog/cat' owner are sharing  some common feature or the cat/dog owner are also a fish owner. 

In [67]:
Percentage_of_user_100=ad1.withColumn('percentage',ad1['cat_dog']/ad1['total']).sort('percentage',ascending=False).  \
                                 filter(ad1['total']<=50)

In [68]:

number_rank=50

print('Creator_name with number of userid less than 50')
print('The top {} creator_name are {}.'.format(number_rank,Percentage_of_user_100.limit(number_rank).show()))

Creator_name with number of userid less than 50
+--------------------+-----+-------+----------+
|        creator_name|total|cat_dog|percentage|
+--------------------+-----+-------+----------+
|         Big Grizzly|    2|    2.0|       1.0|
|          steve dale|    1|    1.0|       1.0|
|          SDReptiles|    1|    1.0|       1.0|
|                K945|    5|    5.0|       1.0|
|Piano Tutorials b...|    8|    8.0|       1.0|
|       MrAnimal Farm|    8|    8.0|       1.0|
|      Labrador Daisy|    3|    3.0|       1.0|
|          DogIDs.com|    4|    4.0|       1.0|
|        AndrewConboy|    3|    3.0|       1.0|
|        LeMasterReef|   13|   13.0|       1.0|
|Aphmau X anime fo...|   11|   11.0|       1.0|
|     Jonette Crowley|    6|    6.0|       1.0|
|          nubianjuan|    1|    1.0|       1.0|
|             CBleezy|    8|    8.0|       1.0|
|   Best Bully Sticks|    2|    2.0|       1.0|
|                Rach|    5|    5.0|       1.0|
|     Georgia Trapper|    8|    8.0|    

For creator_name have less than 50 userid. It only have one creator_name 'DogIDs.com' related with dog/cat. While 
literally, it seems other creator_name are not related with dog/cat topic.

In [69]:
end=time.time()

In [70]:
print('The running time is {0:.2f} hours.'.format((end-start)/3600))

The running time is 3.35 hours.
